IMPORT LIBRARIES

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os

SETUP WEBDRIVER

In [6]:
option = webdriver.ChromeOptions()
option.add_argument('--headless')
option.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36')

service = Service('../chromedriver-win32/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=option)

driver.set_window_size(1300, 800)

ACCESS WEB

In [7]:
keyword = 'sepatu'
URL = f'https://www.tokopedia.com/search?st=&q={keyword}'

driver.get(URL)

pixel = 1000    # size scroll in pixel
for i in range(1, 7):   # 6 x scroll
    scroll = pixel * i
    js = f'window.scrollTo(0, {str(scroll)})'
    driver.execute_script(js)
    time.sleep(1)

time.sleep(5)
driver.save_screenshot('screenshot.png')

pageSource = driver.page_source
driver.quit()

SCRAPING

In [15]:
content = BeautifulSoup(pageSource, 'html.parser')
items = content.findAll('div', 'pcv3__container css-gfx8z3')

imageList, nameList, priceList, locationList, shopList, ratingList, soldList, linkList = [], [], [], [], [], [], [], []
i = 1

for item in items:
    image = item.find('img', 'css-1q90pod')['src']
    name = item.find('div', 'prd_link-product-name css-3um8ox').text
    price = item.find('div', 'prd_link-product-price css-1ksb19c').text
    location = item.find('span', 'prd_link-shop-loc css-1kdc32b flip')
    shop = item.find('span', 'prd_link-shop-name css-1kdc32b flip')
    rating = item.find('span', 'prd_rating-average-text css-t70v7i')
    sold = item.find('span', 'prd_label-integrity css-1duhs3e')
    link = item.find('a', 'pcv3__info-content css-gwkf0u')['href']

    if location: location = location.text
    if shop: shop = shop.text
    if rating: rating = rating.text
    if sold: sold = sold.text

    imageList.append(image)
    nameList.append(name)
    priceList.append(price)
    locationList.append(location)
    shopList.append(shop)
    ratingList.append(rating)
    soldList.append(sold)
    linkList.append(link)

    res = requests.get(image)
    if res.status_code == 200:
        image_data = res.content
        image_name = ''.join(filter(str.isalnum, name))
        image_filename = os.path.join('images', f'{image_name}.jpg')
        with open(image_filename, 'wb') as f:
            f.write(image_data)
            i += 1

DATAFRAME

In [16]:
data = {'Product Name': nameList, 'Price': priceList, 'Location': locationList, 'Shop Name': shopList,
        'Rating': ratingList, 'Sold': soldList, 'Image': imageList, 'Link': linkList}

dataframe = pd.DataFrame(data)
dataframe.head()

,Product Name,Price,Location,Shop Name,Rating,Sold,Image,Link
0,"Sepatu wanita sneakers import 007 - maroon, 39",Rp 89.000,None,None,4.4,Terjual 8,https://images.tokopedia.net/img/cache/200-squ...,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...
1,Sepatu sneakers pria import 00003,Rp 149.000,None,None,4.7,Terjual 22,https://images.tokopedia.net/img/cache/200-squ...,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...
2,Sepatu pria sneakers 009 import,Rp 129.900,None,None,5.0,Terjual 11,https://images.tokopedia.net/img/cache/200-squ...,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...
3,Loro Piona Mules Suede Sepatu Wanita VVIP Qual...,Rp1.300.000,Jakarta Utara,moca_fashionluxe,5.0,Terjual 23,https://images.tokopedia.net/img/cache/200-squ...,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...
4,Sepatu NB 550 LDW Green - BB550ALD - 40,Rp785.000,Jakarta Barat,AUF SHOES,5.0,Terjual 5,https://images.tokopedia.net/img/cache/200-squ...,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...


In [17]:
# Export to CSV
dataframe.to_csv('Data Products.csv', index=False)